# BitMEX Wallets in/outflow
Date: 5/20/20  
Author: Evan Azevedo  
Organization: Amberdata  

## Description
Exploring the question: observing the in/outflows from BitMEX wallets, is there some impact on price? 

## Research Question
Does the inflow of money into BitMEX wallets have any correlation with the price?

### Key information
* All wallet addresses start with BMEX (I think those are motor bikes?)
* BitMEX have a single daily withdraw. So these wallets can take a lot of deposits, but limited withdraw - similar to a High Yield Savings acct.

In [117]:
# import the packages
import os
import requests
import json
import datetime
import matplotlib.pyplot as plt
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import tqdm

N = 30 # Number of days to consider

In [75]:
def get_key():
    "Get the API key from an .env file"
    if ".env" not in os.listdir("../"):
        print("Configuring API Key...")
        key = input("Amberdata API Key: ")
        with open(".env", "w") as f:
            f.write(f"AMBERDATA_API_KEY={key}\n")

    load_dotenv(verbose=True)

    return {
        "AMBERDATA_API_KEY": os.getenv("AMBERDATA_API_KEY"),
        "AWS_ACCESS_KEY": os.getenv("AWS_ACCESS_KEY"),
        "AWS_SECRET_KEY": os.getenv("AWS_SECRET_KEY")
    }

api_key = get_key()["AMBERDATA_API_KEY"]

In [76]:
def get_response(url, headers=None, queryString=None):
    "Get the REST response from the specified URL"
    if not headers:
        headers = {'x-api-key': api_key}
    if queryString:
        response = requests.request("GET", url, headers=headers, params=queryString)
    else:
        response = requests.request("GET", url, headers=headers)
    response = json.loads(response.text)
    if response["title"] == "OK":
        return response["payload"]
    else:
        print("Unsuccessful request.")

In [105]:
def inflow_outflow(data: dict):
    "Returns the inflow and outflow of the payload"
    # get the column names
    columns = data["metadata"]["columns"]
    # load the data, dropping timestampNano
    ad_hist = pd.DataFrame(data["data"], columns=columns).drop("timestampNanoseconds", axis=1)
    # change dtype of appropriate columns to Int
    ad_hist[["blockNumber", "timestamp", "value"]] = ad_hist[["blockNumber", "timestamp", "value"]].apply(pd.to_numeric)
    # sort by blockNum desc
    ad_hist = ad_hist.sort_values("timestamp").reset_index(drop=True)
    # calculate inflow and outflow
    ad_hist["diff"] = ad_hist["value"].diff()
    ad_hist["inflow"] = np.where(ad_hist["diff"] > 0, ad_hist["diff"], 0)
    ad_hist["outflow"] = np.where(ad_hist["diff"] < 0, abs(ad_hist["diff"]), 0)
    # return the result
    return ad_hist.inflow.values, ad_hist.outflow.values

In [77]:
df = pd.read_excel("/Users/evanazevedo/Downloads/Cluster_addresses_of_Bitmex.xlsx", header=12, skipfooter=3)

In [139]:
endTime = datetime.datetime.now()
startTime = endTime - datetime.timedelta(N)

endTime = str(int(endTime.timestamp()))
startTime = str(int(startTime.timestamp()))

querystring = {"startDate": startTime,
               "endDate": endTime}

headers = {
    'x-amberdata-blockchain-id': "bitcoin-mainnet",
    'x-api-key': api_key
}

index = [10**3*(int(startTime) + i*24*60**2) for i in range(N)]
columns = ["inflow", "outflow"]
gross_daily = pd.DataFrame(index=index, columns=columns)

In [140]:
print(gross_daily)

              inflow outflow
1588037584000    NaN     NaN
1588123984000    NaN     NaN
1588210384000    NaN     NaN
1588296784000    NaN     NaN
1588383184000    NaN     NaN
1588469584000    NaN     NaN
1588555984000    NaN     NaN
1588642384000    NaN     NaN
1588728784000    NaN     NaN
1588815184000    NaN     NaN
1588901584000    NaN     NaN
1588987984000    NaN     NaN
1589074384000    NaN     NaN
1589160784000    NaN     NaN
1589247184000    NaN     NaN
1589333584000    NaN     NaN
1589419984000    NaN     NaN
1589506384000    NaN     NaN
1589592784000    NaN     NaN
1589679184000    NaN     NaN
1589765584000    NaN     NaN
1589851984000    NaN     NaN
1589938384000    NaN     NaN
1590024784000    NaN     NaN
1590111184000    NaN     NaN
1590197584000    NaN     NaN
1590283984000    NaN     NaN
1590370384000    NaN     NaN
1590456784000    NaN     NaN
1590543184000    NaN     NaN


In [143]:
d = np.digitize(ad_hist.timestamp.values, index)

In [149]:
g = ad_hist[["inflow", "outflow"]].groupby(d).sum()
g["net"] = g["inflow"] - g["outflow"]

In [154]:
g

,inflow,outflow,net
0,128140172.0,125164399.0,2975773.0
2,0.0,7780484.0,-7780484.0
3,4199232.0,3102562.0,1096670.0
4,0.0,1058225.0,-1058225.0
5,894838.0,184271.0,710567.0
6,0.0,717370.0,-717370.0
7,7899499.0,0.0,7899499.0
8,5316852.0,7933944.0,-2617092.0
9,1007115.0,5294015.0,-4286900.0
10,2339128.0,465666.0,1873462.0


In [111]:
addresses = df.iloc[:100].Address.values

for address in tqdm.tqdm(addresses):
    url = "https://web3api.io/api/v2/addresses/" + address + "/account-balances/historical"
    payload = get_response(url=url, headers=headers, queryString=querystring)
    if len(payload["data"]) > 1:
        inflow, outflow = inflow_outflow(payload)

100%|██████████| 100/100 [00:51<00:00,  1.92it/s]


In [129]:
inflow

array([      0.,       0., 5942421.,       0.,       0.,  190640.])